# 01 - Simulating Free Particles in One Dimension

**Overview** 

This notebook guides you through one of the simplest systems we can model, a collection of free classical particles (atoms, electrons, molecules) traveling in a one-dimensional simulation space, with particle-particle interactions modelled as reflecting hard-sphere potentials.  

**Problem** 

Let's assume we have a carbon nanotube or an aquaporin protein (aka a water-channel protein) filled with water molecules at arbitrary positions. The water molecules are all identical (same mass) and they all start with the same speed, randomly oriented either up or down the channel. Assume that water molecules move as free particles in one dimension and that they have completely elastic collisions with each other. 

>Given that the channel length is 10 nm, and the speed of the molecules is 1 nm/ns, what is the maximum time that it takes for all the water molecules to get out of the channel?

In the following we will run a simulation with only two particles. To make the simulation results more clear, one of the particles will start at the origin, and the other particle will start in the middle of the channel and will have a speed twice as large as the first particle. Moreover, the simulation has reflecting hard walls, meaning that when a particle hits the ends of the channel it will bounce back. 

**Questions**

Before you run any simulation, answer the following question(s):

1. Consider a simplified problem where you only have one of the two particles at a time. Can you predict and draw a sketch of the trajectory (x vs. t) for these systems?

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

2. The simulation has a few parameters that you can play with explicitly. Which of these parameters is/are connected to the underlying physics of the problem (physical parameters) and which of these is/are instead connected to the numerical simulation (numerical parameters)?  
3. For the physical parameter(s), what experimental observable is connected to the parameters or, alternatively, how would you set their initial values? 
4. For the numerical parameter(s), how do they affect the accuracy of the results? In other words, does the simulation run correctly for all possible values of these parameters? What issues do you find?
5. For the numerical parameter(s), how do they affect the computational cost of the simulation? 
6. How would you decide the best value of the numerical parameters, given the physical properties of the system?
7. The simulation loop is hiding (at least) one more numerical parameter, which is not explicitly exposed to the user. Can you find it and explain what it is?
8. The simulation loop is designed for 2 particles. How many operations (lines of code) apply to each particle? How many operations apply to each pair of particles? If you where to trivially extend the code to N particles, what would be the scaling of the code with N? 

**Notebook Setup** 

In Python programs we will always rely on a variety of features that go beyond basic programming tools. These features are collected in specialized packages called Modules, which we need to import in our script/notebook in order to be able to use them. While we can import these modules at any time in our program, we will try to keep the declarations of modules at the very beginning of every notebook. 

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
# Install Plotly (if not already)
!pip install -q plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# @title Simulation Parameters  { display-mode: "form" }
channel_length = 10.0 # @param {type:"number"}
velocity_scale = 1  # @param {type:"number"}
dt = 0.1  # @param {type:"number"}
nsteps = 1000  # @param {type:"integer"}
frame_stride = 1 # @param {type:"integer"}
total_time = nsteps * dt

In [ ]:
# @title Run and Visualize the Simulation  { display-mode: "form" }
# Initial conditions
position_a = 0.
velocity_a = 1.0 * velocity_scale
position_b = 5.
velocity_b = -2.0 * velocity_scale

trajectory_a = []
trajectory_b = []
time = []

# Run the simulation loop
t = 0
for _ in range(nsteps):
    time.append(t)
    trajectory_a.append(position_a)
    trajectory_b.append(position_b)

    position_a += velocity_a * dt
    position_b += velocity_b * dt

    # Elastic collision check
    if abs(position_b - position_a) < 1e-1:
        velocity_a, velocity_b = velocity_b, velocity_a

    # Reflect at walls
    if position_a > channel_length or position_a < 0:
        velocity_a = -velocity_a
    if position_b > channel_length or position_b < 0:
        velocity_b = -velocity_b

    t += dt

# The following code creates an animated plot using Plotly
# --- Initial figure ---
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=False,
    row_heights=[0.4, 0.6],           # Bottom plot a bit taller
    vertical_spacing=0.20,            # Increase spacing between plots
    subplot_titles=("Particle Positions", "Trajectories")
)

# Initial positions
fig.add_trace(go.Scatter(x=[trajectory_a[0]], y=[0], mode='markers', marker=dict(size=15), name='A'), row=1, col=1)
fig.add_trace(go.Scatter(x=[trajectory_b[0]], y=[0], mode='markers', marker=dict(size=15), name='B'), row=1, col=1)

# Initial trajectory lines
fig.add_trace(go.Scatter(x=[time[0]], y=[trajectory_a[0]], mode='lines', name='A Trajectory',
                         line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=[time[0]], y=[trajectory_b[0]], mode='lines', name='B Trajectory',
                         line=dict(color='red')), row=2, col=1)

# --- Animation frames ---
frames = []

for i in range(frame_stride, nsteps, frame_stride):
    frames.append(go.Frame(
        data=[
            go.Scatter(x=[trajectory_a[i]], y=[0], mode='markers', marker=dict(size=15)),
            go.Scatter(x=[trajectory_b[i]], y=[0], mode='markers', marker=dict(size=15)),
            go.Scatter(x=time[:i], y=trajectory_a[:i], mode='lines', line=dict(color='blue')),
            go.Scatter(x=time[:i], y=trajectory_b[:i], mode='lines', line=dict(color='red')),
        ]
    ))
fig.update_layout(
    height=600,
    title=f"1D Hard Sphere Simulation (velocity_scale={velocity_scale}, dt={dt})",
    
    # Top subplot: positions
    xaxis=dict(range=[0, channel_length], title="Position"),
    yaxis=dict(showticklabels=False),
    
    # Bottom subplot: trajectories
    xaxis2=dict(title="Time", range=[0, nsteps * dt]),       # ✅ full time range
    yaxis2=dict(title="Position", range=[0, channel_length]),           # ✅ full spatial range

    updatemenus=[
        dict(
            type='buttons',
            showactive=False,
            buttons=[
                dict(label='Play',
                     method='animate',
                     args=[None, {
                         "frame": {"duration": 30, "redraw": True},
                         "fromcurrent": True,
                         "transition": {"duration": 0}
                     }])
            ]
        )
    ]
)

# ✅ Don't forget to assign frames
fig.frames = frames
fig.show()

**Homework Assignment**

Pick one (or more) of the following projects:
1. Modify the code to handle vanishing/absorbing boundary coditions 
2. Modify the code to handle periodic boundary conditions
3. Modify the code to handle more than 2 particles
4. Modify the code to run the simulation in 2 dimensions

For the modified code, report a plausible physical system that can be modeled using the code. Report convergence tests on the numerical parameters of the simulation. Report results of multiple simulations as you vary the physical parameters of the system. 

NOTE: It is not necessary that the modified code produces an animation, but you should be able to visualize the results of the simulation in some way. 